# Forecasting

In [6]:
import boto3
import datetime
import io
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.units as munits
import numpy as np
from sklearn.metrics import mean_squared_error

In [7]:
# AWS credentials to read files on S3 bucket
f = open('../credentials.json')
credentials = json.load(f)

s3_client = boto3.client(
    "s3",
    aws_access_key_id=credentials["Access key ID"],
    aws_secret_access_key=credentials["Secret access key"]
    )

s3_resource = boto3.resource(
    "s3",
    aws_access_key_id=credentials["Access key ID"],
    aws_secret_access_key=credentials["Secret access key"]
    )

In [8]:
prefix_objs = s3_resource.Bucket("cge").objects.filter(Prefix="output")
keys = [obj.key for obj in prefix_objs]
keys

['output/',
 'output/cluster_a_gb.csv',
 'output/cluster_a_mlp.csv',
 'output/cluster_a_rid.csv',
 'output/cluster_b_gb.csv',
 'output/cluster_b_mlp.csv',
 'output/cluster_b_rid.csv',
 'output/cluster_c_gb.csv',
 'output/cluster_c_mlp.csv',
 'output/cluster_c_rid.csv',
 'output/cluster_d_gb.csv',
 'output/cluster_d_mlp.csv',
 'output/cluster_d_rid.csv',
 'output/jabaquara_gb.csv',
 'output/jabaquara_mlp.csv',
 'output/jabaquara_rid.csv',
 'output/lapa_gb.csv',
 'output/lapa_mlp.csv',
 'output/lapa_rid.csv',
 'output/parelheiros_gb.csv',
 'output/parelheiros_mlp.csv',
 'output/parelheiros_rid.csv',
 'output/penha_gb.csv',
 'output/penha_mlp.csv',
 'output/penha_rid.csv',
 'output/pirituba_gb.csv',
 'output/pirituba_mlp.csv',
 'output/pirituba_rid.csv',
 'output/se_gb.csv',
 'output/se_mlp.csv',
 'output/se_rid.csv',
 'output_teste_2/cluster_a_rid.csv']

In [9]:
model_name = {
    "rid": "Ridge",
    "mlp": "Multi-layer Perceptron",
    "gb": "Histogram-based Gradient Boosting Regression Tree",
}

# getting evaluation metric from predictions
# prefix_objs = s3_resource.Bucket("cge").objects.filter(Prefix="output")
# keys = [obj.key for obj in prefix_objs]

keys = [
    'output/cluster_a_gb.csv',
    'output/cluster_a_mlp.csv',
    'output/cluster_a_rid.csv',
    'output/cluster_b_gb.csv',
    'output/cluster_b_mlp.csv',
    'output/cluster_b_rid.csv',
    'output/cluster_c_gb.csv',
    'output/cluster_c_mlp.csv',
    'output/cluster_c_rid.csv',
    'output/cluster_d_gb.csv',
    'output/cluster_d_mlp.csv',
    'output/cluster_d_rid.csv',
]

for key in keys:
    cluster_file_name = key.replace("output/", "").replace(".csv", "")
    model = key.replace("output/", "").replace(".csv", "").split("_")[2]
    
    if cluster_file_name.split("_")[1] == "a":
        processed_file_name = "cluster_A"
    elif cluster_file_name.split("_")[1] == "b":
        processed_file_name = "cluster_B"
    elif cluster_file_name.split("_")[1] == "c":
        processed_file_name = "cluster_C"
    elif cluster_file_name.split("_")[1] == "d":
        processed_file_name = "cluster_D"

    obj_true = s3_client.get_object(Bucket="cge", Key=f"processed/{processed_file_name}.csv")
    obj_pred = s3_client.get_object(Bucket="cge", Key=key)

    obj_true_rural = s3_client.get_object(Bucket="cge", Key="processed/parelheiros.csv")
    obj_pred_rural = s3_client.get_object(Bucket="cge", Key=f"output/parelheiros_{model}.csv")
    
    y_true = pd.read_csv(io.BytesIO(obj_true["Body"].read()))
    y_true = y_true[["timestamp", "temperature"]].dropna()
    y_true["timestamp"] = pd.to_datetime(y_true["timestamp"])
    y_true = y_true.set_index("timestamp")

    y_pred = pd.read_csv(io.BytesIO(obj_pred["Body"].read()), index_col=0)
    cluster_name = y_pred.cluster.unique()[0]
    y_pred = y_pred.drop(["cluster"], axis=1)
    y_pred["timestamp"] = pd.to_datetime(y_pred["timestamp"])

    y_true_rural = pd.read_csv(io.BytesIO(obj_true_rural["Body"].read()))
    y_true_rural = y_true_rural[["timestamp", "temperature"]].dropna()
    y_true_rural["timestamp"] = pd.to_datetime(y_true_rural["timestamp"])
    y_true_rural = y_true_rural.set_index("timestamp")

    y_pred_rural = pd.read_csv(io.BytesIO(obj_pred_rural["Body"].read()), index_col=0)
    y_pred_rural = y_pred_rural.drop(["cluster"], axis=1)
    y_pred_rural["timestamp"] = pd.to_datetime(y_pred_rural["timestamp"])

    mse_temperature = []
    mse_uhi = []
    for i in y_pred.index:
        try:
            pred = y_pred.loc[[i]]
            start_date = pred.timestamp.unique()[0] + np.timedelta64(1, "h")
            pred = pred.drop("timestamp", axis=1).T
            pred.index = pd.date_range(start=start_date, periods=6, freq="H")
            pred.columns = ["temperature"]

            pred_rural = y_pred_rural[y_pred_rural.timestamp == start_date]
            pred_rural = pred_rural.drop("timestamp", axis=1).T
            pred_rural.index = pd.date_range(start=start_date, periods=6, freq="H")
            pred_rural.columns = ["temperature_rural"]

            pred_uhii = pred.merge(pred_rural, left_index=True, right_index=True)
            pred_uhii["uhii"] = pred_uhii.temperature - pred_uhii.temperature_rural
            pred_uhii = pred_uhii.drop(["temperature", "temperature_rural"], axis=1)

            test_plot = y_true[pred.index[0]:pred.index[-1]]
            test_plot_rural = y_true_rural[pred.index[0]:pred.index[-1]]
            test_plot_rural.columns = ["temperature_rural"]
            test_plot_uhii = test_plot.merge(test_plot_rural, left_index=True, right_index=True)
            test_plot_uhii["uhii"] = test_plot_uhii.temperature - test_plot_uhii.temperature_rural
            test_plot_uhii = test_plot_uhii.drop(["temperature", "temperature_rural"], axis=1)

            try:
                mse_temperature.append(mean_squared_error(test_plot, pred))
                mse_uhi.append(mean_squared_error(test_plot_uhii, pred_uhii))
            except:
                pass
        except:
            pass

    # temperature evaluation
    model_error_temperature = pd.DataFrame({
        "Cluster": [cluster_file_name.split("_")[1]],
        "Model": [model_name[model]],
        "MSE": [np.round(np.mean(mse_temperature), 2)],
        "MSE standard deviation": [np.round(np.std(mse_temperature), 2)]
        })

    cluster_model = cluster_file_name.split("_")[1] + "_" + model
    buffer = io.StringIO()
    model_error_temperature.to_csv(buffer, index=False)
    s3_resource.Object("cge", f"evaluation/temperature_evaluation/{cluster_model}.csv").put(Body=buffer.getvalue())

    # UHI evaluation        
    model_error_uhi = pd.DataFrame({
        "Cluster": [cluster_file_name.split("_")[1]],
        "Model": [model_name[model]],
        "MSE": [np.round(np.mean(mse_uhi), 2)],
        "MSE standard deviation": [np.round(np.std(mse_uhi), 2)]
        })

    cluster_model = cluster_file_name.split("_")[1] + "_" + model
    buffer = io.StringIO()
    model_error_uhi.to_csv(buffer, index=False)
    s3_resource.Object("cge", f"evaluation/uhi_evaluation/{cluster_model}.csv").put(Body=buffer.getvalue())

In [10]:
prefix_objs = s3_resource.Bucket("cge").objects.filter(Prefix="evaluation/temperature_evaluation")
keys = [obj.key for obj in prefix_objs]
keys

['evaluation/temperature_evaluation/a_gb.csv',
 'evaluation/temperature_evaluation/a_mlp.csv',
 'evaluation/temperature_evaluation/a_rid.csv',
 'evaluation/temperature_evaluation/b_gb.csv',
 'evaluation/temperature_evaluation/b_mlp.csv',
 'evaluation/temperature_evaluation/b_rid.csv',
 'evaluation/temperature_evaluation/c_gb.csv',
 'evaluation/temperature_evaluation/c_mlp.csv',
 'evaluation/temperature_evaluation/c_rid.csv',
 'evaluation/temperature_evaluation/d_gb.csv',
 'evaluation/temperature_evaluation/d_mlp.csv',
 'evaluation/temperature_evaluation/d_rid.csv',
 'evaluation/temperature_evaluation/jabaquara_gb.csv',
 'evaluation/temperature_evaluation/jabaquara_mlp.csv',
 'evaluation/temperature_evaluation/jabaquara_rid.csv',
 'evaluation/temperature_evaluation/lapa_gb.csv',
 'evaluation/temperature_evaluation/lapa_mlp.csv',
 'evaluation/temperature_evaluation/lapa_rid.csv',
 'evaluation/temperature_evaluation/parelheiros_gb.csv',
 'evaluation/temperature_evaluation/parelheiros_mlp.

In [11]:
dfs = []

keys = [
    'evaluation/temperature_evaluation/a_gb.csv',
    'evaluation/temperature_evaluation/a_mlp.csv',
    'evaluation/temperature_evaluation/a_rid.csv',
    'evaluation/temperature_evaluation/b_gb.csv',
    'evaluation/temperature_evaluation/b_mlp.csv',
    'evaluation/temperature_evaluation/b_rid.csv',
    'evaluation/temperature_evaluation/c_gb.csv',
    'evaluation/temperature_evaluation/c_mlp.csv',
    'evaluation/temperature_evaluation/c_rid.csv',
    'evaluation/temperature_evaluation/d_gb.csv',
    'evaluation/temperature_evaluation/d_mlp.csv',
    'evaluation/temperature_evaluation/d_rid.csv',
]

for key in keys:
    obj = s3_client.get_object(Bucket="cge", Key=key)
    dfs.append(pd.read_csv(io.BytesIO(obj["Body"].read())))

evaluation = pd.concat(dfs).reset_index(drop=True)
evaluation

,Cluster,Model,MSE,MSE standard deviation
0,a,Histogram-based Gradient Boosting Regression Tree,5.76,8.82
1,a,Multi-layer Perceptron,6.47,9.77
2,a,Ridge,6.37,11.90
3,b,Histogram-based Gradient Boosting Regression Tree,5.62,7.98
4,b,Multi-layer Perceptron,6.85,10.03
5,b,Ridge,6.36,9.62
6,c,Histogram-based Gradient Boosting Regression Tree,4.90,7.49
7,c,Multi-layer Perceptron,5.17,13.57
8,c,Ridge,5.59,17.96
9,d,Histogram-based Gradient Boosting Regression Tree,5.64,8.74


In [12]:
prefix_objs = s3_resource.Bucket("cge").objects.filter(Prefix="evaluation/uhi_evaluation")
keys = [obj.key for obj in prefix_objs]
keys

['evaluation/uhi_evaluation/a_gb.csv',
 'evaluation/uhi_evaluation/a_mlp.csv',
 'evaluation/uhi_evaluation/a_rid.csv',
 'evaluation/uhi_evaluation/b_gb.csv',
 'evaluation/uhi_evaluation/b_mlp.csv',
 'evaluation/uhi_evaluation/b_rid.csv',
 'evaluation/uhi_evaluation/c_gb.csv',
 'evaluation/uhi_evaluation/c_mlp.csv',
 'evaluation/uhi_evaluation/c_rid.csv',
 'evaluation/uhi_evaluation/d_gb.csv',
 'evaluation/uhi_evaluation/d_mlp.csv',
 'evaluation/uhi_evaluation/d_rid.csv',
 'evaluation/uhi_evaluation/jabaquara_gb.csv',
 'evaluation/uhi_evaluation/jabaquara_mlp.csv',
 'evaluation/uhi_evaluation/jabaquara_rid.csv',
 'evaluation/uhi_evaluation/lapa_gb.csv',
 'evaluation/uhi_evaluation/lapa_mlp.csv',
 'evaluation/uhi_evaluation/lapa_rid.csv',
 'evaluation/uhi_evaluation/parelheiros_gb.csv',
 'evaluation/uhi_evaluation/parelheiros_mlp.csv',
 'evaluation/uhi_evaluation/parelheiros_rid.csv',
 'evaluation/uhi_evaluation/penha_gb.csv',
 'evaluation/uhi_evaluation/penha_mlp.csv',
 'evaluation/uhi_

In [13]:
dfs = []

keys = [
    'evaluation/uhi_evaluation/a_gb.csv',
    'evaluation/uhi_evaluation/a_mlp.csv',
    'evaluation/uhi_evaluation/a_rid.csv',
    'evaluation/uhi_evaluation/b_gb.csv',
    'evaluation/uhi_evaluation/b_mlp.csv',
    'evaluation/uhi_evaluation/b_rid.csv',
    'evaluation/uhi_evaluation/c_gb.csv',
    'evaluation/uhi_evaluation/c_mlp.csv',
    'evaluation/uhi_evaluation/c_rid.csv',
    'evaluation/uhi_evaluation/d_gb.csv',
    'evaluation/uhi_evaluation/d_mlp.csv',
    'evaluation/uhi_evaluation/d_rid.csv'
]

for key in keys:
    obj = s3_client.get_object(Bucket="cge", Key=key)
    dfs.append(pd.read_csv(io.BytesIO(obj["Body"].read())))

evaluation = pd.concat(dfs).reset_index(drop=True)
evaluation

,Cluster,Model,MSE,MSE standard deviation
0,a,Histogram-based Gradient Boosting Regression Tree,3.82,7.31
1,a,Multi-layer Perceptron,5.01,8.77
2,a,Ridge,4.51,10.15
3,b,Histogram-based Gradient Boosting Regression Tree,3.62,6.90
4,b,Multi-layer Perceptron,4.88,8.45
5,b,Ridge,4.41,8.33
6,c,Histogram-based Gradient Boosting Regression Tree,2.79,5.40
7,c,Multi-layer Perceptron,3.59,12.32
8,c,Ridge,3.68,18.05
9,d,Histogram-based Gradient Boosting Regression Tree,2.88,5.87


In [14]:
evaluation.loc[evaluation.groupby("Cluster")["MSE"].idxmin()]

,Cluster,Model,MSE,MSE standard deviation
0,a,Histogram-based Gradient Boosting Regression Tree,3.82,7.31
3,b,Histogram-based Gradient Boosting Regression Tree,3.62,6.90
6,c,Histogram-based Gradient Boosting Regression Tree,2.79,5.40
9,d,Histogram-based Gradient Boosting Regression Tree,2.88,5.87


In [15]:
prefix_objs = s3_resource.Bucket("cge").objects.filter(Prefix="output")
keys = [obj.key for obj in prefix_objs]
keys

['output/',
 'output/cluster_a_gb.csv',
 'output/cluster_a_mlp.csv',
 'output/cluster_a_rid.csv',
 'output/cluster_b_gb.csv',
 'output/cluster_b_mlp.csv',
 'output/cluster_b_rid.csv',
 'output/cluster_c_gb.csv',
 'output/cluster_c_mlp.csv',
 'output/cluster_c_rid.csv',
 'output/cluster_d_gb.csv',
 'output/cluster_d_mlp.csv',
 'output/cluster_d_rid.csv',
 'output/jabaquara_gb.csv',
 'output/jabaquara_mlp.csv',
 'output/jabaquara_rid.csv',
 'output/lapa_gb.csv',
 'output/lapa_mlp.csv',
 'output/lapa_rid.csv',
 'output/parelheiros_gb.csv',
 'output/parelheiros_mlp.csv',
 'output/parelheiros_rid.csv',
 'output/penha_gb.csv',
 'output/penha_mlp.csv',
 'output/penha_rid.csv',
 'output/pirituba_gb.csv',
 'output/pirituba_mlp.csv',
 'output/pirituba_rid.csv',
 'output/se_gb.csv',
 'output/se_mlp.csv',
 'output/se_rid.csv',
 'output_teste_2/cluster_a_rid.csv']

In [16]:
model_name = {
    "rid": "Ridge",
    "mlp": "Multi-layer Perceptron",
    "gb": "Histogram-based Gradient Boosting Regression Tree",
}

# getting plots from predictions

keys = [
    'output/cluster_a_gb.csv',
    'output/cluster_a_mlp.csv',
    'output/cluster_a_rid.csv',
    'output/cluster_b_gb.csv',
    'output/cluster_b_mlp.csv',
    'output/cluster_b_rid.csv',
    'output/cluster_c_gb.csv',
    'output/cluster_c_mlp.csv',
    'output/cluster_c_rid.csv',
    'output/cluster_d_gb.csv',
    'output/cluster_d_mlp.csv',
    'output/cluster_d_rid.csv',
]

for key in keys:
    cluster_file_name = key.replace("output/", "").replace(".csv", "")
    model = key.replace("output/", "").replace(".csv", "").split("_")[2]
    
    if cluster_file_name.split("_")[1] == "a":
        processed_file_name = "cluster_A"
    elif cluster_file_name.split("_")[1] == "b":
        processed_file_name = "cluster_B"
    elif cluster_file_name.split("_")[1] == "c":
        processed_file_name = "cluster_C"
    elif cluster_file_name.split("_")[1] == "d":
        processed_file_name = "cluster_D"

    # if model == "gb":
    obj_true = s3_client.get_object(Bucket="cge", Key=f"processed/{processed_file_name}.csv")
    obj_pred = s3_client.get_object(Bucket="cge", Key=key)

    obj_true_rural = s3_client.get_object(Bucket="cge", Key="processed/parelheiros.csv")
    obj_pred_rural = s3_client.get_object(Bucket="cge", Key=f"output/parelheiros_{model}.csv")
    
    y_true = pd.read_csv(io.BytesIO(obj_true["Body"].read()))
    y_true = y_true[["timestamp", "temperature"]].dropna()
    y_true["timestamp"] = pd.to_datetime(y_true["timestamp"])
    y_true = y_true.set_index("timestamp")

    y_pred = pd.read_csv(io.BytesIO(obj_pred["Body"].read()), index_col=0)
    cluster = y_pred.cluster.unique()[0]
    y_pred = y_pred.drop(["cluster"], axis=1)
    y_pred["timestamp"] = pd.to_datetime(y_pred["timestamp"])

    y_true_rural = pd.read_csv(io.BytesIO(obj_true_rural["Body"].read()))
    y_true_rural = y_true_rural[["timestamp", "temperature"]].dropna()
    y_true_rural["timestamp"] = pd.to_datetime(y_true_rural["timestamp"])
    y_true_rural = y_true_rural.set_index("timestamp")

    y_pred_rural = pd.read_csv(io.BytesIO(obj_pred_rural["Body"].read()), index_col=0)
    y_pred_rural = y_pred_rural.drop(["cluster"], axis=1)
    y_pred_rural["timestamp"] = pd.to_datetime(y_pred_rural["timestamp"])

    converter = mdates.ConciseDateConverter()
    munits.registry[np.datetime64] = converter
    munits.registry[datetime.date] = converter
    munits.registry[datetime.datetime] = converter

    for i in y_pred.index[:500]:
        try:
            pred = y_pred.loc[[i]]
            start_date = pred.timestamp.unique()[0] + np.timedelta64(1, "h")
            pred = pred.drop("timestamp", axis=1).T
            pred.index = pd.date_range(start=start_date, periods=6, freq="H")
            pred.columns = ["temperature"]

            pred_rural = y_pred_rural[y_pred_rural.timestamp == start_date]
            pred_rural = pred_rural.drop("timestamp", axis=1).T
            pred_rural.index = pd.date_range(start=start_date, periods=6, freq="H")
            pred_rural.columns = ["temperature_rural"]

            pred_uhii = pred.merge(pred_rural, left_index=True, right_index=True)
            pred_uhii["uhii"] = pred_uhii.temperature - pred_uhii.temperature_rural
            pred_uhii = pred_uhii.drop(["temperature", "temperature_rural"], axis=1)

            past_data = 24 * 2
            train_plot = y_true[pred.index[0]-np.timedelta64(past_data, "h"):pred.index[0]-np.timedelta64(1, "h")]
            train_plot_rural = y_true_rural[pred.index[0]-np.timedelta64(past_data, "h"):pred.index[0]-np.timedelta64(1, "h")]
            train_plot_rural.columns = ["temperature_rural"]
            train_plot_uhii = train_plot.merge(train_plot_rural, left_index=True, right_index=True)
            train_plot_uhii["uhii"] = train_plot_uhii.temperature - train_plot_uhii.temperature_rural
            train_plot_uhii = train_plot_uhii.drop(["temperature", "temperature_rural"], axis=1)

            test_plot = y_true[pred.index[0]:pred.index[-1]]
            test_plot_rural = y_true_rural[pred.index[0]:pred.index[-1]]
            test_plot_rural.columns = ["temperature_rural"]
            test_plot_uhii = test_plot.merge(test_plot_rural, left_index=True, right_index=True)
            test_plot_uhii["uhii"] = test_plot_uhii.temperature - test_plot_uhii.temperature_rural
            test_plot_uhii = test_plot_uhii.drop(["temperature", "temperature_rural"], axis=1)

            fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(9,9), sharex=True)
            fig.suptitle(f"Cluster: {cluster.upper()} / Model: {model_name[model]}")
            
            # urban temperature prediction
            ax1.plot(train_plot, label=f"Observado (últimas {past_data} horas)", marker=".", markersize=8, alpha=0.8)
            ax1.plot(test_plot, label="Teste", ls="", marker=".", markersize=8, color="forestgreen", alpha=0.8)
            ax1.plot(pred, label="Previsão", ls="", marker="X", markersize=5, color="orangered", alpha=0.8)
            ax1.set_ylabel("Temp. urbana (°C)")
            ax1.legend()
            ax1.grid(alpha=0.3)

            # rural temperature prediction
            ax2.plot(train_plot_rural, label=f"Observado (últimas {past_data} horas)", marker=".", markersize=8, alpha=0.8)
            ax2.plot(test_plot_rural, label="Teste", ls="", marker=".", markersize=8, color="forestgreen", alpha=0.8)
            ax2.plot(pred_rural, label="Previsão", ls="", marker="X", markersize=5, color="orangered", alpha=0.8)
            ax2.set_ylabel("Temp. rural (°C)")
            ax2.legend()
            ax2.grid(alpha=0.3)
            
            # UHI prediction
            ax3.plot(train_plot_uhii, label=f"Observado (últimas {past_data} horas)", marker=".", markersize=8, alpha=0.8)
            ax3.plot(test_plot_uhii, label="Teste", ls="", marker=".", markersize=8, color="forestgreen", alpha=0.8)
            ax3.plot(pred_uhii, label="Previsão", ls="", marker="X", markersize=5, color="orangered", alpha=0.8)
            ax3.set_ylabel("IICU (°C)")
            ax3.legend()
            ax3.grid(alpha=0.3)

            img_data = io.BytesIO()
            plt.savefig(img_data, format="png", dpi=300, bbox_inches="tight")
            img_data.seek(0)
            bucket = s3_resource.Bucket("cge")
            station_model_dt = cluster + "_" + model + "_" + str(pred_uhii.index[0]).replace("-", "").replace(" ", "").replace(":", "")
            bucket.put_object(Body=img_data, ContentType="image/png", Key=f"figures/{cluster}/{model}/{station_model_dt}")
            plt.close()
        except:
            pass